In [3]:
import os
from functools import partial

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from statsforecast.models import AutoETS
from statsmodels.tsa.seasonal import seasonal_decompose

from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mae, mape, mase
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, Markdown, Image

from fpppy.utils import plot_series

# endregion 
from plotnine import *

# change working directory 
# Check if current directory has any .ipynb files
if any(fname.endswith(".ipynb") for fname in os.listdir()):
    os.chdir("..")

# load local package modules

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1.3. Some simpler time series models

Technically, a **time series model** provides a complete probabilistic model  that specifies all the joint distributions of random vectors ($X_1, ..., X_n$). That is, the probability at each *n* time point that the variable ($X_n$) would be less than or equal to any possible value of $- \infty < x_1, ..., x_n < \infty$. Mathematically, 

$$
P[X_1 \le x_1,..., X_n \le x_n], \infty < x_1, ..., x_n < \infty. n=, 1,2,...
$$

Because this is not practically feasible, only the mean and variance of the joint distributions are modelled (i..e, second-order properties). The theory of minimium mean sqare errors justifies use of second-order properties and guarantees that little information is lost. 

In [ ]:
### 